# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 1:32PM,238554,19,ADV80002988,"AdvaGen Pharma, Ltd",Released
1,Jun 1 2022 1:28PM,238553,16,SAR00000337,Sartorius Stedim North America,Released
2,Jun 1 2022 1:16PM,238552,102,PBR-051822-04,"Senseonics, Incorporated",Released
3,Jun 1 2022 12:58PM,238551,19,GP06062022,"Granules Pharmaceuticals, Inc.",Released
4,Jun 1 2022 12:57PM,238542,10,0085732280,ISDIN Corporation,Released
5,Jun 1 2022 12:57PM,238542,10,0085732281,ISDIN Corporation,Released
6,Jun 1 2022 12:57PM,238542,10,0085728041,ISDIN Corporation,Released
7,Jun 1 2022 12:57PM,238542,10,0085732419,ISDIN Corporation,Released
8,Jun 1 2022 12:57PM,238542,10,0085732416,ISDIN Corporation,Released
9,Jun 1 2022 12:57PM,238542,10,0085732420,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,238549,Released,1
47,238551,Released,1
48,238552,Released,1
49,238553,Released,1
50,238554,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238549,NaN,NaN,1.0
238551,NaN,NaN,1.0
238552,NaN,NaN,1.0
238553,NaN,NaN,1.0
238554,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0.0,13.0,7.0
238343,0.0,0.0,1.0
238433,0.0,8.0,15.0
238436,0.0,0.0,2.0
238438,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0,13,7
238343,0,0,1
238433,0,8,15
238436,0,0,2
238438,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,0,13,7
1,238343,0,0,1
2,238433,0,8,15
3,238436,0,0,2
4,238438,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,,13,7
1,238343,,,1
2,238433,,8,15
3,238436,,,2
4,238438,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd"
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated"
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc."
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation
50,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.
77,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc."
78,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions
79,Jun 1 2022 12:48PM,238546,10,Beach Products Inc
97,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd",,,1
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America,,,1
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated",,,1
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",,,1
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,,,46
5,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,,,27
6,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",,,1
7,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,,,1
8,Jun 1 2022 12:48PM,238546,10,Beach Products Inc,,,18
9,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc.",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America,1,,
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated",1,,
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1,,
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46,,
5,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,27,,
6,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",1,,
7,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,1,,
8,Jun 1 2022 12:48PM,238546,10,Beach Products Inc,18,,
9,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc.",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America,1,,
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated",1,,
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1,,
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated",1.0,NaN,NaN
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 1:32PM,238554,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jun 1 2022 1:28PM,238553,16,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 1 2022 1:16PM,238552,102,"Senseonics, Incorporated",1.0,0.0,0.0
3,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
4,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2385120,109.0,49.0,9.0
102,954024,4.0,0.0,0.0
12,477037,60.0,2.0,0.0
15,1430558,47.0,52.0,3.0
16,1431228,7.0,0.0,0.0
18,476865,2.0,0.0,0.0
19,1192674,8.0,12.0,12.0
20,1192586,69.0,4.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2385120,109.0,49.0,9.0
1,102,954024,4.0,0.0,0.0
2,12,477037,60.0,2.0,0.0
3,15,1430558,47.0,52.0,3.0
4,16,1431228,7.0,0.0,0.0
5,18,476865,2.0,0.0,0.0
6,19,1192674,8.0,12.0,12.0
7,20,1192586,69.0,4.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,109.0,49.0,9.0
1,102,4.0,0.0,0.0
2,12,60.0,2.0,0.0
3,15,47.0,52.0,3.0
4,16,7.0,0.0,0.0
5,18,2.0,0.0,0.0
6,19,8.0,12.0,12.0
7,20,69.0,4.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,102,Released,4.0
2,12,Released,60.0
3,15,Released,47.0
4,16,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,9.0,0.0,0.0,3.0,0.0,0.0,12.0,6.0
Executing,49.0,0.0,2.0,52.0,0.0,0.0,12.0,4.0
Released,109.0,4.0,60.0,47.0,7.0,2.0,8.0,69.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,9.0,0.0,0.0,3.0,0.0,0.0,12.0,6.0
1,Executing,49.0,0.0,2.0,52.0,0.0,0.0,12.0,4.0
2,Released,109.0,4.0,60.0,47.0,7.0,2.0,8.0,69.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,9.0,0.0,0.0,3.0,0.0,0.0,12.0,6.0
1,Executing,49.0,0.0,2.0,52.0,0.0,0.0,12.0,4.0
2,Released,109.0,4.0,60.0,47.0,7.0,2.0,8.0,69.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()